In [10]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime as dt
import base64
import os as OS

In [12]:
from pythonopensubtitles.opensubtitles import OpenSubtitles
os = OpenSubtitles()

token = os.login('mradu22@gmail.com', 'romaniaa11')

In [14]:
print(token)

None


In [14]:
def search_best_sub(imdb_id_search):
    #imdb_id_search='1055369'
    data = os.search_subtitles([{'sublanguageid': 'eng','imdbid': imdb_id_search,'SubFormat': 'srt'}])
    assert type(data) == list


    #search and select the highest rated subtitle based on user download (maybe rating too) file and store its ID
    best=[]
    for i in range(0,len(data)):
        best.append(float(data[i]['SubRating'])*float(data[i]['SubDownloadsCnt']))  
    best_sub_id=data[best.index(max(best))]['IDSubtitleFile']  
    print('Found the best one', best_sub_id)
    return data

def download_sub(best_sub_id):
    #############################################
    ##crucial line as you only have 200 per day##
    #subdata=os.download_subtitles([best_sub_id])
    ##crucial line as you only have 200 per day##
    #############################################
    return subdata

def decode_store(subdata):
    #decode and upzip the file
    compressed_subdata = decode(bytearray(subdata[0]['data'].encode("utf-8")),'base64')
    sub_text = gzip.GzipFile(fileobj=io.BytesIO(compressed_subdata)).read()

    #parse it and store it into a DataFrame with three columns, starttime, endtime, and line
    sub_text_str = str(sub_text)
    snew = sub_text_str.replace("\\n","\n").replace("\\r","\r").replace("</i>","").replace("<i>-","").replace("<i>","").replace("\\","")
    subs=list(filter(None, snew.split('\r\n')))
    subs=[x for x in subs if not x.isdigit()]
    if subs[0][0]=='b':
        subs.pop(0)

    starttimes=[]
    endtimes=[]
    words=[]
    #"-->" in line
    prevLineWasWords =False
    for line in subs:
        line=line.strip()
        if "-->" in line:
            starttimes.append(line[:8])
            endtimes.append(line[17:25])
        elif len(line) > 1:
                if prevLineWasWords:
                        words[len(words)-1] = words[len(words)-1] + " " + line
                else:
                        words.append(line)
                prevLineWasWords = True
                continue
        prevLineWasWords = False  

    #this needs to be made better but for now we need these to be same length so it will have to do
    trim_by=min([len(starttimes),len(endtimes),len(words)])
    starttimes=starttimes[0:trim_by]
    endtimes=endtimes[0:trim_by]
    words=words[0:trim_by]

    plines=pd.DataFrame()
    s1 = pd.Series(starttimes)
    s2 = pd.Series(endtimes)
    s3 = pd.Series(words)
    plines['StartTime'] = s1.values
    plines['EndTime'] = s2.values
    plines['Line'] = s3.values
    return plines




In [17]:
data=os.search_subtitles([{'sublanguageid': 'eng','imdbid': 1055369,'SubFormat': 'srt'}])

In [20]:
print(data)

None


In [70]:
BData=dict()
mov_list=['0120338','2322441','0109830','0398808','0119643']
for ii in range(0,len(mov_list)):
    BData[mov_list[ii]] = decode_store(download_sub(search_best_sub(mov_list[ii])))

Found the best one 1953045324
Found the best one 1954649943
Found the best one 1951637834
Found the best one 1951669715
Found the best one 1951641851


In [71]:
for ii in range(0,len(mov_list)):
    if len(BData[mov_list[ii]])==0:
        BData.pop(mov_list[ii],None)
Sub_data = pd.Panel(BData)
Sub_data.items

Index(['0109830', '0119643', '0120338', '0398808', '2322441'], dtype='object')

In [72]:
fname1='DraDB'
fname=fname1+'.h5'
if not OS.path.isfile(fname):
    Sub_data.to_hdf(fname, 'key',append=True, min_itemsize=50)
else:
    Sub_data.to_hdf(fname, 'key',append=True, min_itemsize=50)

In [14]:
imdb_ids=[item['imdb_id'] for item in movie_database]

In [18]:
import collections
print([item for item, count in collections.Counter(imdb_ids).items() if count > 1])

[None, '']


In [20]:
movie_database[324]

{'adult': False,
 'backdrop_path': '/T3LrH6bnV74llVbFpQsCBrGaU9.jpg',
 'belongs_to_collection': None,
 'budget': 90000000,
 'genres': [{'id': 12, 'name': 'Adventure'},
  {'id': 28, 'name': 'Action'},
  {'id': 37, 'name': 'Western'}],
 'homepage': 'http://www.sonypictures.com/movies/themagnificentseven/',
 'id': 333484,
 'imdb_id': 'tt2404435',
 'keywords': {'keywords': [{'id': 9714, 'name': 'remake'}]},
 'original_language': 'en',
 'original_title': 'The Magnificent Seven',
 'overview': 'Seven gun men in the old west gradually come together to help a poor village against savage thieves.',
 'popularity': 35.846722,
 'poster_path': '/z6BP8yLwck8mN9dtdYKkZ4XGa3D.jpg',
 'production_companies': [{'id': 5, 'name': 'Columbia Pictures'},
  {'id': 79, 'name': 'Village Roadshow Pictures'},
  {'id': 1423, 'name': 'Escape Artists'},
  {'id': 8411, 'name': 'Metro-Goldwyn-Mayer (MGM)'},
  {'id': 34034, 'name': 'LStar Capital'},
  {'id': 79566, 'name': 'Pin High Productions'}],
 'production_countries

In [26]:
sorted([(item['budget'],item['title']) for item in movie_database], reverse=True)

[(500000000, 'Justice League'),
 (500000000, 'Avengers: Infinity War'),
 (380000000, 'Pirates of the Caribbean: On Stranger Tides'),
 (300000000, "Pirates of the Caribbean: At World's End"),
 (280000000, 'Avengers: Age of Ultron'),
 (270000000, 'Superman Returns'),
 (260000000, 'Tangled'),
 (260000000, 'John Carter'),
 (258000000, 'Spider-Man 3'),
 (255000000, 'The Lone Ranger'),
 (250000000, 'X-Men: Days of Future Past'),
 (250000000, 'The Hobbit: The Desolation of Smaug'),
 (250000000, 'The Hobbit: The Battle of the Five Armies'),
 (250000000, 'The Hobbit: An Unexpected Journey'),
 (250000000, 'The Fate of the Furious'),
 (250000000, 'The Dark Knight Rises'),
 (250000000, 'Harry Potter and the Half-Blood Prince'),
 (250000000, 'Harry Potter and the Deathly Hallows: Part 1'),
 (250000000, 'Batman v Superman: Dawn of Justice'),
 (245000000, 'Star Wars: The Force Awakens'),
 (245000000, 'Spectre'),
 (242875078, '8 Mile'),
 (240000000, 'Captain America: Civil War'),
 (237000000, 'Avatar'